In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.models import load_model,Model

Using TensorFlow backend.


In [0]:
df=pd.read_csv("/content/drive/My Drive/FER/fer2013.csv")

In [5]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
X_train,y_train,X_test,y_test=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [0]:
num_features = 64
num_labels = 7
width, height = 48, 48

In [0]:
X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

y_train=np_utils.to_categorical(y_train, num_classes=num_labels)
y_test=np_utils.to_categorical(y_test, num_classes=num_labels)

In [0]:
#normalizing data between o and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48 ,48, 1)

In [12]:
# Architecture of the CNN Model

model = Sequential()

# 1st Convolution layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

# 2nd Convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# 3rd Convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))


model.add(Flatten())

# Fully Connected layer
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 64)         

In [0]:
#Compliling the model
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])

In [16]:
#Training the model
model.fit(X_train, y_train,batch_size=256,epochs=200,verbose=1, validation_data=(X_test, y_test),shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/200
28709/28709 [==============================] - 10s 336us/step - loss: 2.1894 - acc: 0.2170 - val_loss: 1.8193 - val_acc: 0.2494
Epoch 2/200
28709/28709 [==============================] - 8s 284us/step - loss: 1.9576 - acc: 0.2380 - val_loss: 1.8114 - val_acc: 0.2491
Epoch 3/200
28709/28709 [==============================] - 8s 286us/step - loss: 1.8940 - acc: 0.2515 - val_loss: 1.8120 - val_acc: 0.2516
Epoch 4/200
28709/28709 [==============================] - 8s 289us/step - loss: 1.8757 - acc: 0.2505 - val_loss: 1.8650 - val_acc: 0.2555
Epoch 5/200
28709/28709 [==============================] - 8s 290us/step - loss: 1.8517 - acc: 0.2669 - val_loss: 1.8708 - val_acc: 0.2536
Epoch 6/200
28709/28709 [==============================] - 8s 294us/step - loss: 1.8128 - acc: 0.2894 - val_loss: 1.7213 - val_acc: 0.2881
Epoch 7/200
28709/28709 [==============================] - 8s 292us/step - loss: 1.8162 - acc: 0.2989 - val_loss: 1.

In [0]:
model.save("/content/drive/My Drive/Facial_Emotion_Detector.h5")

In [0]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("/content/drive/My Drive/Emotion_Detector_Model.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("/content/drive/My Drive/Emotion_Detector_Model_Weights.h5")